## Enap - Machine Learning em Projetos

### Projeto Final

# Análise de microdados do Enem: diferenças de rendimento entre alunos de escolas públicas e privadas

**Autor:** Stefano Mozart Pontes Canedo de Souza.

**Objetivo:**

O objetivo deste projeto é utilizar as técnicas apresentadas no curso para analisar possíveis diferenças de rendimento entre escolas públicas e privadas presentes nos microdados do Enem. A intuição, baseada em todo o histórico da educação pública no Brasil, é que o rendimento dos egressos do ensino público seja, na média, inferior àquele observado entre alunos de escolas privadas. 

**Método:**

O método de análise empregado contará com a construção de dois grupos de modelos de classificação: o primeiro, tentando classificar o aluno como egresso de escola pública ou privada, a partir de suas notas. O segundo grupo, parte de informações socieconômicas, incluindo a classificação da escola, para prever a nota esperada para um dado aluno.

O que se deseja demonstrar com esses modelos é que, caso os dados de fato apresentem uma diferença significativa nos resultados obtidos por alunos de escolas públicas e privadas, essas diferenças se refletirão no poder de classificação dos modelos treinados a partir desses dados.

## Análise exploratória

Os dados utilizados neste experimento são disponibilizados pelo Inep em http://inep.gov.br/web/guest/microdados. Para os modelos de classificação de alunos, foram utilizados os dados mais recentes, do Enem 2019. Para os modelos de classificação de escolas, foram utilizados os dados dos três últimos anos disponíveis: 2019, 2018 e 2017.

Para o primeiro grupo de modelos, usamos as seguintes propriedades do registro:


|Propriedade      | Descrição                             |
|:--------------- |:--------------------------------------|
| TP_ESCOLA	      | Tipo de escola do Ensino Médio <sup>1</sup>|
| NU_NOTA_CN      |Nota da prova de Ciências da Natureza  |
| NU_NOTA_CH	  |Nota da prova de Ciências Humanas      |
| NU_NOTA_LC	  |Nota da prova de Linguagens e Códigos  |
| NU_NOTA_MT	  |Nota da prova de Matemática            |
| NU_NOTA_REDACAO |Nota da prova de redação               |

Para o segundo grupo, foram utilizadas as seguintes propriedades:

|Propriedade     | Descrição                              |
|:---------------|:-------------------------------------- |
| TP_ESCOLA	     | Tipo de escola do Ensino Médio         |
| TP_ENSINO	     | Tipo de instituição que concluiu ou concluirá o Ensino Médio|
| SG_UF_ESC	     | Sigla da Unidade da Federação da escola|
| TP_SEXO	     | Sexo                                   |
| TP_COR_RACA	 | Cor/raça                               |
| Q001	         | Formação acadêmica do pai              |
| Q002           | Formação acadêmica da mãe              |
| Q005           | Número de pessoas na residência        |
| Q006           | Faixa de renda familiar mensal         |
| Q007           | Família contrata empregada doméstica   |
| NOTA_MEDIA     | Média das notas objetivas <sup>2</sup> |
| NU_NOTA_REDACAO| Nota da redação                        |

**Observações:**

1. Os valores possíveis para esse campo são: 1. Não informou; 2. Pública; 3. Privada; 4. Exterior. Utilizamos nesta análise apenas os registros que apresetnem valor 1 ou 2 nesta propriedade.
2. A *feature* NOTA_MEDIA é calculada da seguinte forma: NOTA_MEDIA = (NU_NOTA_CN + NU_NOTA_CH + NU_NOTA_LC + NU_NOTA_MT) / 4


In [ ]:
!pip install pingouin

In [ ]:
# Carregamos as bibliotecas que serão utilizadas para manipulação e visualização dos dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pingouin as pg
import seaborn as sns

In [ ]:
# Criamos um pipeline de pré-processamento. A ideia é utilizar essa função para microdados de
# diferentes anos 
def pipeline_notas_Enem(arquivo):
    # Colunas a serem lidas no arquivo
    features = [
        'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT','NU_NOTA_REDACAO','TP_ESCOLA'
    ]
    
    # Lemos o arquivo, retirando os registros em que um dos valores não estivesse presente.
    df = pd.read_csv(
        arquivo,
        #nrows = 5000, # 5k linhas para desenvolvimento inicial
        encoding = 'latin1',
        usecols = features,
        sep = ';'
    ).dropna()
    
    df['NOTA_MEDIA'] = (df['NU_NOTA_CN'] + df['NU_NOTA_CH'] + df['NU_NOTA_LC'] + df['NU_NOTA_MT']) / 4
    
    # Filtramos os registros de com alunos de escolas públicas e privadas (valores 2 e 3 no 
    # campo TP_ESCOLA)
    df = df.loc[df['TP_ESCOLA'].isin([2, 3])]
    df.loc[df['TP_ESCOLA']==2, 'TP_ESCOLA'] = 'Pública'
    df.loc[df['TP_ESCOLA']==3, 'TP_ESCOLA'] = 'Privada'
    
    return df

# Carregamos o dataset a partir da cópia do Kaggle
notas = pipeline_notas_Enem('/kaggle/input/enem-2019/DADOS/MICRODADOS_ENEM_2019.csv')

notas.head()

### Distribuição entre as classes

Ao analisarmos a distribuição de alunos entre os dois grupos, tanto no dataset original quanto nos subconjuntos de teste e treinamento, percebemos que o dataset é bastante desbalanceado. Cerca de 83% dos elementos da amostra pertencem à classe majoritária: alunos de escolas públicas. Esse desbalanceamento será levado em conta na análise dos modelos a seguir.

Para garantir a capacidade de comparação entre modelos, separamos o dataset em treinamento em teste, numa proporção de 80%/20%.

In [ ]:
notas.TP_ESCOLA.value_counts().plot(kind='bar')
plt.show()

### Diferenças entre as classes

Ao analizarmos, por exemplo, a nota da redação, podemos concluir que existe uma diferença estatisticamente significante entre os dois grupos

In [ ]:
notas[['TP_ESCOLA', 'NU_NOTA_REDACAO']].groupby('TP_ESCOLA').describe()

In [ ]:
pub = notas.loc[notas.TP_ESCOLA=='Pública', 'NU_NOTA_REDACAO']
priv = notas.loc[notas.TP_ESCOLA=='Privada', 'NU_NOTA_REDACAO']

import warnings
warnings.filterwarnings('ignore')

pg.ttest(priv, pub)

O teste nos mostra que a diferença entre as médias é estatisticamente significativa. O poder do teste é 1. Abaixo, vemos os histogramas das duas classes.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 3))

p1=sns.distplot(
    pub,
    ax=axes[0],
    axlabel=f'Média: {pub.mean():.2f}\nDesvio padrao: {pub.std():.2f}'
).set_title("Notas de alunos de escola pública")
p2=sns.distplot(
    priv,
    axlabel=f'Média: {priv.mean():.2f}\nDesvio padrao: {priv.std():.2f}'
).set_title("Notas de alunos de escola privada")
plt.show()

O mesmo se aplica à nota média nas provas objetivas:

In [ ]:
notas[['TP_ESCOLA', 'NOTA_MEDIA']].groupby('TP_ESCOLA').describe()

In [ ]:
pub = notas.loc[notas.TP_ESCOLA=='Pública', 'NOTA_MEDIA']
priv = notas.loc[notas.TP_ESCOLA=='Privada', 'NOTA_MEDIA']
pg.ttest(x=priv, y=pub, correction=False).round(2)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 3))

p1=sns.distplot(
    pub,
    ax=axes[0],
    axlabel=f'Média: {pub.mean():.2f}\nDesvio padrao: {pub.std():.2f}'
).set_title("Nota de alunos de escola pública")
p2=sns.distplot(
    priv,
    axlabel=f'Média: {priv.mean():.2f}\nDesvio padrao: {priv.std():.2f}'
).set_title("Nota de alunos de escola privada")
plt.show()

### Pré processamento

Agora, realizamos a normalização dos dados, a fim de acelerarmos o treinamento e teste dos modelos preditivos

In [ ]:
std_features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'NOTA_MEDIA']
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
notas[std_features] = std.fit_transform(notas[std_features])

E, para fins de comparação dos modelos, separamos o dataset em treinamento em teste, numa proporção de 80%/20%


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    notas.drop(['TP_ESCOLA', 'NOTA_MEDIA'], axis = 1), notas.TP_ESCOLA, test_size=0.2, random_state=42
)

## Classificando alunos a partir de suas notas

Para o primeiro grupo de classificadores, utilizaremos as notas dos alunos, bem como sua classe de renda no questionário sócio-econômico, para classificar se o aluno é egresso de escola pública ou privada. 

O primeiro classificador utilizado é o KNN, que agrupa ocorrências da amostra a partir de um critério de similaridade. O hiper-parâmetro mais importante desse modelo é justamente o número de núcleos a partir dos quais o modelo realizará o agrupamento. Nesse caso, utilizamos dois núcleos, seguindo a intuição de que há dois grupos distintos no *dataset*: alunos egressos de escolas públicas e privadas.

In [ ]:
# Instanciamos o modelo KNN com dois núcleos
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)

# Definimos uma função para treinamento e exbição dos resultados de um modelo
def pipeline_treino_teste(model):
    # Ajustamos o modelo
    model.fit(X_train, y_train)
    # Submetemos os dados de teste ao classificador 
    y_pred = model.predict(X_test)
    
    # E observamos algumas métricas de desempenho desse modelo: acurácia, F1-score e matriz de confusão
    from sklearn import metrics
    print(f'Acurácia: {metrics.accuracy_score(y_test, y_pred)}')
    print(f'F1-score médio: {metrics.f1_score(y_test, y_pred, average="weighted")}')
    print(f"F1-score da classe minoritária: {metrics.f1_score(y_test, y_pred, pos_label='Privada')}")
    metrics.plot_confusion_matrix(model, X_test, y_test, cmap=plt.cm.Blues, normalize='true')
    
# Agora executamos o pipeline
pipeline_treino_teste(knn)

O resultado acima, com acurácia de cerca de 83%, que é a proporção de elementos da classe majoritária (escola pública), com score F1 da classe minoritária (privada) abaixo de 30%, indicam que, muito provavelmente, o classificador KNN é simples demais para modelar o problema proposto. O poder de classificação modelado ainda não apresenta evidencia forte o suficiente da diferença de rendimento entre alunos de escolas públicas e privadas. A matriz de confusão acima, por sua vez, indica que o modelo erra bastante dado ao viés da classe majoritária. Prosseguimos, então, nosso experimento com mais 3 classificadores.

O próximo modelo utilizado é o de descida de gradiente estocástica (SGD), selecionamos a função de custo (loss) do tipo 'modified_huber', que resulta num modelo de regressão linear que é mais robusto contra outliers. A função de custo (loss) do tipo "elasticnet" favorece a seleção de variáveis (feature selection) durante o próprio treinamento. Tendo em vista o desbalanceamento entre as classes, utilizaremos o parâmetro class_weight="balanced", que pondera a função de erro por um fator inversamente proporcional à participação da classe na amostra.

In [ ]:
# Instanciamos um modelo SGD (descida do gradiente estocástica)
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(class_weight="balanced", loss='modified_huber', penalty="elasticnet", random_state=42)

# Executamos o pipeline de treino e teste
pipeline_treino_teste(sgd)

Embora apresente um score f1 melhor para a classe minoritária, e tenha acurácia melhor para essa classe (a matriz de confusão nos mostra 75% de acerto na classe minoritária), a acurácia ponderada para todo a amostra é baixa, tendo em vista o percentual de 26% erros na classe majoritária.

O próximo modelo utilizado será o de regressão logística.

In [ ]:
# Instanciamos o modelo de regressão logística
from sklearn.linear_model import LogisticRegression
rlog = LogisticRegression(class_weight="balanced")

# E submetemos ao pipeline de treino e teste
pipeline_treino_teste(rlog)

O desempenho do modelo de regressão logística é ligeiramente superior ao do modelo anterior. A matriz de confusão acima, no entanto não apresenta uma redução significativa no viés de classificação. Cerca de 26% das ocorrências da classe minoritária foram classificadas incorretamente.

A seguir, testaremos um modelo de árvore de decisão.

In [ ]:
# Instanciamos uma árvore de decisão, com altura máxima de 3 nós
from sklearn.tree import DecisionTreeClassifier, plot_tree
tree = DecisionTreeClassifier(class_weight="balanced", max_depth=7, random_state=42)

# Executamos o pipeline de treino e teste
pipeline_treino_teste(tree)

O modelo de árvore de decisão teve desempenho muito similar ao do modelo de regressão logística, em ambas as classes. Mais uma vez, é importante notar que a acurácia de quase 75% se deve ao fato de que o modelo erra na classificação de cerca um quarto dos registros da classe majoritária.

O próximo modelo utilizado é o RandomForest, que cria, internamente, uma série de árvores de decisão e apresenta, como resultado de classificação, uma combinação das respostas apresentadas pelas diversas árvores treinadas com os dados.

In [ ]:
# Instanciamos o classificador Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(class_weight="balanced", n_estimators=300, random_state=42)

# Executamos o pipeline de treino e teste
pipeline_treino_teste(rf)

O resultado desse classificador é, na verdade, uma composição de resultados dos 300 estimadores criados internamente. É interessante notar que, embora haja uma diferença significativa na acurácia, a melhora se dá apenas na classe majoritária: agora, com apenas 3% de erros. O modelo erra, no entanto, em cerca de 72% das ocorrências da classe minoritária.

A seguir, testamos um modelo de máquinas de vetores-suporte. Trata-se de um modelo que busca maximizar as fronteiras de separação entre classes. Neste experimento, utilizamos a versão linear do classificador, pois o custo de computação da versão não linear (SVC) é quadrático em relação ao número de registros na amostra, o que o torna inviável para um dataset com mais de 1 milhão de registros.

In [ ]:
# Instanciamos o classificador SVM
from sklearn.svm import LinearSVC
lsvm = LinearSVC(class_weight="balanced", max_iter=3000, random_state=42, tol=5e-4)

# Submetemos o classificador ao treino e teste
pipeline_treino_teste(lsvm)

Esse classificador tem desempenho ligeriamente superior ao dos modelos de regressão logística e árvore de decisão. Sua acurácia, assim como nos classificadores anteriores, é afetada pela porcentagem de erros na classe majoritária.

O próximo classificador utilizado é o XGBoost (eXtreme Gradient Boosting), que utiliza diversos modelos de árvore de decisão como estimadores subjacentes, mas faz uma otimização da busca por hiperparâmetros ótimos, de acordo com a tarefa selecionada. Nesse experimento, utilizamos o parâmetro objective='multi:softmax' para que modelo otimize os parâmetros para classificação.

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(
    objective = 'multi:softmax',
    booster = 'gbtree',
    num_class = 2,
    eval_metric = 'logloss',
    eta = .1,
    max_depth = 14,
    colsample_bytree = .4,
    n_jobs=-1
)

pipeline_treino_teste(xgb)

Esse modelo tem acurácia maior, mas, assim como o modelo de Random Forest, esse ganho se dá como resultado de um viés de classificação para a classe majoritária. Quando observamos a matriz de confusão, percebemos que o modelo erra em quase 70% dos registros da classe minoritária.

Por fim, usaremos um classificador que utiliza os modelos já treinados anteriormente como estimadores subjacentes para gerar uma classificação própria. Neste caso, atribuímos pesos iguais para todos modelos e o resultado da classificação será dado pela maioria simples de votos. É possível testar diferentes combinações de pesos e alcançar um desempenho superior.

In [ ]:
# Instanciamos o classficador por votos, passando os classificadores já treinados
from mlxtend.classifier import EnsembleVoteClassifier

vote = EnsembleVoteClassifier(
    clfs=[knn, sgd, rlog, tree, rf, lsvm, xgb],
    weights=[1, 1, 1, 1, 1, 1, 1],
    refit=False
)

# Submetemos esse classificador ao teste
pipeline_treino_teste(vote)

O classificador final tem acurácia semelhante a dos modelos de Random Forest e XGBoost, mas com desempenho superior quando consideramos sua atuação na classe minoritária.

Esse resultado nos mostra que os modelos de fato captam uma diferença nas notas de alunos de escolas públicas e privadas. Todos os modelos foram instanciados com a maior parte dos hiperparâmetros em seu valor padrão. É possível que a aplicação de técnicas de otimização de hiperparâmetros produza modelos ainda melhores.

### Utilizando um modelo mais complexo

Os modelos utilizados até aqui são considerados modelos "clássicos" de Machine Learning. A maior parte desses modelos prioriza a simplicidade e explicabilidade dos resultados. Mas é possível que modelos de Deep Learning, mais compexos e mais difíceis de analizar, apresentem resultados superiores em termos de acurácia.

A seguir, utilizamos a biblioteca fast.ai, que utiliza embbedings e outras técnicas avançadas para seleção, ajuste e treinamento de modelos.

In [ ]:
from fastai.tabular import *

dep_var = 'TP_ESCOLA'
cont_names = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'NOTA_MEDIA']

start = int(len(notas)*.7)
end = int(len(notas)*.1) + start

              
test = TabularList.from_df(notas.iloc[start:end], cont_names=cont_names)

data = (
    TabularList
        .from_df(notas, cont_names=cont_names)
        .split_by_idx(list(range(start,end)))
        .label_from_df(cols=dep_var)
        .add_test(test)
        .databunch()
)

data.show_batch(rows=10)

In [ ]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(1, 5e-3)

In [ ]:
ClassificationInterpretation.from_learner(learn).plot_confusion_matrix(normalize=True)

###  Simplificando o problema

A seguir, testamos alguns dos modelos utilizados anteriormente num problemas mais simples: reduzimos as dimensões do dataset, utilizando apenas a nota média das provas objetivas e a nota da redação. Isso nos permite visualizar a dispersão dos registros num plano.

In [ ]:
nX = notas[['NOTA_MEDIA', 'NU_NOTA_REDACAO']]
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
ny = le.fit_transform(notas.TP_ESCOLA)

In [ ]:
from mlxtend.plotting import plot_decision_regions
import matplotlib.gridspec as gridspec
import itertools

gs = gridspec.GridSpec(2, 2)
fig = plt.figure(figsize=(10,8))

labels = ['K-nearest Neighbour', 'Stochastic Gradient Descent', 'Logistic Regression', 'Decision Tree']

for clf, lab, grd in zip([knn, sgd, rlog, tree],
                         labels,
                         itertools.product([0, 1], repeat=2)):
    clf.fit(nX, ny)
    ax = plt.subplot(gs[grd[0], grd[1]])
    fig = plot_decision_regions(X=nX.to_numpy(), y=ny, clf=clf)
    plt.title(lab)

## Prevendo a nota dos alunos a partir de seus dados socioeconômicos

In [ ]:
# Criamos um pipeline de pré-processamento. A ideia é utilizar essa função para microdados de
# diferentes anos 
def pipeline_SocioEconomico_Enem(arquivo):
    # Colunas a serem lidas no arquivo
    features = [
        'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO',  'TP_ESCOLA', 
        'TP_ENSINO', 'SG_UF_ESC', 'TP_COR_RACA', 'TP_SEXO', 'Q001', 'Q002', 'Q005', 'Q006', 'Q007',
        'NU_IDADE'
    ]

    # Carregamos o dataset a partir do arquivo
    df = pd.read_csv(
        arquivo,
        nrows = 5000, # 5k linhas para desenvolvimento inicial
        encoding = 'latin1',
        usecols = features,
        sep = ';'
    )#.dropna()
    
    # Filtramos os registros de com alunos de escolas públicas e privadas (valores 2 e 3 no campo TP_ESCOLA)
    df = df.loc[df['TP_ESCOLA'].isin([2, 3])]
    df.loc[df['TP_ESCOLA']==2, 'TP_ESCOLA'] = 'Pública'
    df.loc[df['TP_ESCOLA']==3, 'TP_ESCOLA'] = 'Privada'

    # Vamos atribuir o tipo de ensino com base na idade do aluno
    df = df.loc[df['NU_IDADE'].notna()]
    df.loc[df['TP_ENSINO'].isna() & df['NU_IDADE']>21, 'TP_ENSINO'] = 3
    df.loc[df['TP_ENSINO'].isna(), 'TP_ENSINO'] = 1
    
    # Filtramos os demais valores ausentes
    df.dropna(inplace=True)
    
    # Realizamos uma normalização dos valores das notas
    std_features = ['NOTA_MEDIA', 'NU_NOTA_REDACAO']
    df['NOTA_MEDIA'] = (df['NU_NOTA_CN'] + df['NU_NOTA_CH'] + df['NU_NOTA_LC'] + df['NU_NOTA_MT']) / 4
    
    
    from sklearn.preprocessing import StandardScaler
    std = StandardScaler()
    df[std_features] = std.fit_transform(df[std_features])
    
    # Usamos um encoder ordinal para transformar os valores presentes na coluna 'Q006' (faixa de renda) em valores 
    # numéricos, crescentes.
    ord_enc_features = ['Q001', 'Q002', 'Q001']
    from sklearn.preprocessing import OrdinalEncoder
    ord_enc = OrdinalEncoder()
    df[ord_enc_features] = ord_enc.fit_transform(df[ord_enc_features])

    ## - Colunas que passarão por um processo de codificação
    #onehot_enc_features = ['TP_COR_RACA', 'TP_SEXO', 'TP_ENSINO', 'SG_UF_ESC']
    #from sklearn.preprocessing import OneHotEncoder
    #onehot_enc = OneHotEncoder()
    #df.enc = onehot_enc.fit_transform(df[onehot_enc_features])

    # Retira as colunas usadas para cálculos intermediários
    return df.drop(['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT'], axis = 1)

# Carregamos o dataset a partir da cópia do Kaggle, retirando os registros em que um dos valores não estivesse presente.
se = pipeline_SocioEconomico_Enem('./dados/MICRODADOS_ENEM_2019.csv')

se.head()

Utilizamos um encoder ordinal, para as variáveis em que há uma gradação de valores categóricos


In [ ]:
ord_enc_features = ['Q005', 'Q006', 'Q007']
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
se[ord_enc_features] = enc.fit_transform(se[ord_enc_features])
se.head()

Criamos variáveis dummies para as demais colunas categórcas

In [ ]:
onehot_enc_features = ['TP_ESCOLA', 'TP_COR_RACA', 'TP_SEXO', 'TP_ENSINO', 'SG_UF_ESC']
se = pd.get_dummies(se, prefix=onehot_enc_features, columns=onehot_enc_features, drop_first=True)
se.head()

### Prevendo a média das provas objetivas

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    se.drop(['NOTA_MEDIA'], axis = 1), se.NOTA_MEDIA, test_size=0.2, random_state=42
)

Nossa variável alvo, nesse experimento será a nota médias nas provas objetivas. Com o dataset subdividido em treino e teste, ajustamos um modelo de regressão linear.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, lr.predict(X_test))

### Prevendo a nota da redação

Alteramos a variável-alvo e repetimos o processo de particionamento do dataset.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    se.drop(['NU_NOTA_REDACAO'], axis = 1), se.NU_NOTA_REDACAO, test_size=0.2, random_state=42
)

lr2 = LinearRegression()
lr2.fit(X_train, y_train)

mean_squared_error(y_test, lr.predict(X_test))

Alteramos a variável-alvo e 